In [2]:
import pandas as pd
from sqlalchemy import create_engine, text, engine
import os
savedir=os.getcwd()+'\\Source_data\\'

In [5]:
df_basic_PlayerStats = pd.read_parquet(f'{savedir}\\df_basic_PlayerStats.gzip')
df_basic_TeamStats = pd.read_parquet(f'{savedir}\\df_basic_TeamStats.gzip')
df_basic_TeamStarterBenchStats = pd.read_parquet(f'{savedir}\\df_basic_TeamStarterBenchStats.gzip')
df_adv_PlayerStats = pd.read_parquet(f'{savedir}\\df_adv_PlayerStats.gzip')
df_adv_TeamStats = pd.read_parquet(f'{savedir}\\df_adv_TeamStats.gzip')

In [11]:
df_adv_TeamStats.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
index,,,,,,,,,,,,,,,,,,,,,
0,0029600001,1610612741,Bulls,CHI,Chicago,240:00,108.0,111.5,99.9,103.2,...,19.8,0.574,0.607,1.0,0.198,98.60,95.5,79.58,96,0.646
1,0029600001,1610612738,Celtics,BOS,Boston,240:00,99.9,103.2,108.0,111.5,...,18.9,0.500,0.521,1.0,0.198,98.60,95.5,79.58,95,0.354
2,0029600002,1610612751,Nets,NJN,New Jersey,240:00,92.2,91.7,107.4,108.4,...,26.2,0.474,0.531,1.0,0.199,83.66,83.5,69.58,84,0.393
3,0029600002,1610612739,Cavaliers,CLE,Cleveland,240:00,107.4,108.4,92.2,91.7,...,18.1,0.507,0.557,1.0,0.200,83.66,83.5,69.58,83,0.607
4,0029600003,1610612755,76ers,PHI,Philadelphia,240:00,104.2,106.2,111.7,114.4,...,14.4,0.458,0.521,1.0,0.200,99.10,97.0,80.83,97,0.467


In [12]:
db_name ='NBA_DB'
db_user_name='Tarun'
db_pwd = 'scsFEwMbbaCmqhlw'
db_host= '34.73.178.171'
#db_host='winter-cab-264200:us-east1:nba-database1'
db_port='3306'


In [13]:
# The SQLAlchemy engine will help manage interactions, including automatically
# managing a pool of connections to your database
db = create_engine(
    # Equivalent URL:
    # postgres+pg8000://<db_user>:<db_pass>@/<db_name>?unix_sock=/cloudsql/<cloud_sql_instance_name>/.s.PGSQL.5432
    engine.url.URL(
        drivername='mysql+pymysql',
        username=db_user_name,
        password=db_pwd,
        database=db_name,
        host=db_host,
        port=db_port
    ),
    # ... Specify additional properties here.
    # ...
)

In [14]:
conn_mysql = 'mysql+pymysql://' + db_user_name + ':' + db_pwd + '@' + db_host + ':' + db_port + '/' + db_name


In [26]:
db=create_engine(conn_mysql)

In [28]:
db.connect()
db.dispose()

In [18]:
with db.connect() as conn:
    df_basic_PlayerStats.to_sql('basic_player_stats',conn,if_exists='append',chunksize=25_000)
    
    df_basic_TeamStats.to_sql('basic_team_stats',conn,if_exists='append',chunksize=25_000)
    
    df_basic_TeamStarterBenchStats.to_sql('basic_team_starter_bench_stats',conn,if_exists='append',chunksize=25_000)
    
    df_adv_PlayerStats.to_sql('adv_player_stats',conn,if_exists='append',chunksize=25_000)
    
    df_adv_TeamStats.to_sql('adv_team_stats',conn,if_exists='append',chunksize=25_000)

In [22]:
with db.connect() as conn:
    seg=pd.read_sql_query('select * from basic_player_stats where PLAYER_NAME = %s limit 200',conn,params=('LeBron James',))

In [24]:
seg

,index,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,521878,0020300014,1610612739,CLE,Cleveland,2544,LeBron James,G,,42:50,...,2.0,4.0,6.0,9.0,4.0,0.0,2.0,3.0,25.0,-9.0
1,521973,0020300018,1610612739,CLE,Cleveland,2544,LeBron James,G,,40:21,...,2.0,10.0,12.0,8.0,1.0,0.0,7.0,1.0,21.0,-3.0
2,522454,0020300038,1610612739,CLE,Cleveland,2544,LeBron James,G,,39:10,...,0.0,4.0,4.0,6.0,2.0,0.0,2.0,3.0,8.0,-21.0
3,522919,0020300057,1610612739,CLE,Cleveland,2544,LeBron James,G,,41:06,...,2.0,9.0,11.0,7.0,2.0,3.0,2.0,1.0,7.0,-3.0
4,523171,0020300068,1610612739,CLE,Cleveland,2544,LeBron James,G,,43:44,...,0.0,5.0,5.0,3.0,0.0,0.0,7.0,2.0,23.0,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,592735,0020500559,1610612739,CLE,Cleveland,2544,LeBron James,F,,41:35,...,2.0,9.0,11.0,9.0,0.0,0.0,2.0,5.0,24.0,-3.0
196,593121,0020500575,1610612739,CLE,Cleveland,2544,LeBron James,F,,35:12,...,1.0,8.0,9.0,6.0,0.0,0.0,6.0,3.0,14.0,-9.0
197,593287,0020500582,1610612739,CLE,Cleveland,2544,LeBron James,F,,42:47,...,0.0,5.0,5.0,8.0,2.0,0.0,3.0,4.0,51.0,19.0
198,593731,0020500600,1610612739,CLE,Cleveland,2544,LeBron James,F,,35:22,...,0.0,7.0,7.0,8.0,2.0,2.0,1.0,0.0,23.0,33.0
